In [184]:
import pandas as pd
import numpy as np
import unidecode
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)
import re

In [185]:
def drop_columns(df, columns):
    df.drop(columns,axis=1, inplace=True)
    return df

In [186]:
def get_furniture(row, furnitures):
#     file = open("links.txt", "r", encoding='utf-8')
#     furnitures = file.read().split('\n')
#     file.close()
    
#     for i,f in enumerate(furnitures):
#         furnitures[i] = unidecode.unidecode(f.lower())
    
    for f in furnitures:
        if f in row[1]:
            return f
    return 'otro'
    
    

In [187]:
df = pd.read_csv("descripcion_productos.csv", encoding='utf-8', sep=";")

In [192]:
# variables
columns = list(df.columns)
l = ['sku','descripcion','img_ambiente', 'img','Material de las patas',
     'Materiales','Construcción de la estructura','Forma del producto',
     'Material principal','Color principal','Año de publicación','Colores',
     'Estilo principal','Estancias']

# the order of the categories matter!!
furnitures = ['abridor', 'alfombra', 'almohada', 'ambientador', 'aparado',
       'aplique', 'armario', 'asiento', 'azucarero', 'bandeja', 'ban',
       'base', 'biombo', 'bolsa', 'bol', 'bombilla', 'book',
       'bote', 'botell', 'brazo', 'butaca', 'cabecero', 'caja',
       'cajon', 'cama', 'cambiador', 'canape', 'candelabro', 'cest', 'chaise',
       'cojin', 'colchoneta', 'colchon', 'colgador', 'consola', 'copa', 
       'corona', 'cortina', 'cuadro', 'cubo', 'delantal', 'escalera', 
       'escobilla', 'escritorio', 'espejo', 'estanteria', 'estante', 'estrella', 
       'figura', 'ganchos', 'guirnalda', 'hamaca', 'jabon', 'jarra', 'jarron', 
       'lampara', 'libreria', 'lienzo', 'llitera', 'macet', 'manta', 
       'mantel', 'marco', 'mecedora', 'mesa', 'mesita', 'modulo', 'mortero',
       'mueble', 'neceser', 'panel', 'pantalla', 'papel', 'perchero', 'pizarra', 
       'planta', 'plato', 'portacepillo', 'protector', 'puf', 'reloj',
       'reposacabezas', 'reposapies', 'revistero', 'rodillo', 'sabana',
       'salero', 'set', 'silla', 'sillon', 'sofacama', 'sofa',
       'somier', 'tabla', 'taburete', 'tapiceria', 'tapiz', 'taza',
       'tetera', 'tipi', 'toall', 'tumbona', 'vaso', 'vela', 'ventosas',
       'zapatero']

file = open("links.txt", "r", encoding='utf-8')
url = file.read().split('\n')
file.close()
    
for i,f in enumerate(url):
    url[i] = unidecode.unidecode(f.lower())
url.remove('chaiselongue')
url.remove('cesto')
url.remove('puff')

url = url + ['chaise', 'cest', 'puf']

# ban -> banco, banqueta
# macet -> macetas, macetero
# cest -> cesta, cestas, cestos
# toall -> toalla, toallas, toallero
# jarr -> jarra, jarron

In [193]:
# get columns of interest
for x in l:
    columns.remove(x)
df = drop_columns(df, columns)

# drop rows with a lot of NA
df.dropna(how='all', subset=['Material de las patas','Materiales','Construcción de la estructura',
                             'Forma del producto','Material principal','Color principal',
                             'Colores','Estilo principal','Estancias'], inplace=True)
df.dropna(subset=['descripcion'], inplace=True)

# lowercase
df['descripcion'] = df['descripcion'].str.lower()
df['descripcion'] = df.apply(lambda x: unidecode.unidecode(x[1]), axis=1)

# add label
df['id'] = df.apply(lambda x: get_furniture(x, furnitures), axis=1)

In [196]:
df_output = df[df['id'].isin(url)].reset_index(drop=True)

In [205]:
df_output.to_json("kavehome_clean.json", orient='records')